In [5]:
import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
tokenizer = RobertaTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

def generate_summary(text):
   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)
   
# text = "Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."

# generate_summary(text)

# Output: L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou.

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [6]:
from summarizer import Summarizer
import os
import textwrap
import time

files = os.listdir('files')

# Create a BERT extractive summarizer
summarizer = Summarizer()

for i, each in enumerate(files):
    print(f"{i+1}/{str(len(files))}: starting summary for {each}...")

    t1 = time.time()

    with open(f'files/{each}', 'r') as file:
        content = file.read()

    # Generate the summary
    # summary = summarizer(content, min_length=50, max_length=200) 
    summary = generate_summary(content)

    wrapped_text = textwrap.fill(summary, width=80)

    # Save the wrapped text to a file
    with open(f'summary/{each}', 'w') as file:
        file.write(wrapped_text)
    
    t2 = time.time()

    print(f"finished in {round(t2-t1, 3)}s!")

## summarisation hugging face model : https://huggingface.co/mrm8488/camembert2camembert_shared-finetuned-french-summarization

1/200: starting summary for PigaultLebrun_Adele.txt...
finished in 4.25s!
2/200: starting summary for Genlis_Emigres.txt...
finished in 4.508s!
3/200: starting summary for Dorat_Malheurs.txt...
finished in 2.683s!
4/200: starting summary for Quesne_Folies.txt...
finished in 5.526s!
5/200: starting summary for Abbes_Voyage.txt...
finished in 3.33s!
6/200: starting summary for Charriere_Observations.txt...
finished in 3.058s!
7/200: starting summary for Loaisel_Comtesse.txt...
finished in 5.487s!
8/200: starting summary for Retif_Paysanne.txt...
finished in 4.29s!
9/200: starting summary for Arnaud_Selicourt.txt...
finished in 4.16s!
10/200: starting summary for Marchadier_Isle.txt...
finished in 3.759s!
11/200: starting summary for Diderot_Jacques.txt...
finished in 3.119s!
12/200: starting summary for Arnaud_Batilde.txt...
finished in 4.058s!
13/200: starting summary for Rousseau_Julie.txt...
finished in 4.926s!
14/200: starting summary for Sade_Infortunes.txt...
finished in 4.208s!
15

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ac0hik/Sentiment_Analysis_French")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("ac0hik/Sentiment_Analysis_French")

## sentiment analysis hugging face model : https://huggingface.co/ac0hik/Sentiment_Analysis_French

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
import pandas as pd

df = pd.read_csv('INDH_dataset.tsv', sep='\t')

for i, each in enumerate(files):
    print(f"{i+1}/{str(len(files))}: starting sentiment analysis for {each}...")

    title = each.split(".txt")[0]

    t1 = time.time()

    with open(f'summary/{each}', 'r') as file:
        content = file.read()
    
    inputs = tokenizer(content, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        logits = sentiment_model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    sentiment = sentiment_model.config.id2label[predicted_class_id]

    # Add prediction label to the DataFrame
    df.loc[df['filename'] == title, 'summary'] = content
    df.loc[df['filename'] == title, 'sentiment'] = sentiment
    
    t2 = time.time()

    print(f"finished in {round(t2-t1, 3)}s!")

df.to_csv("sentiment_output.csv")

1/200: starting sentiment analysis for PigaultLebrun_Adele.txt...
finished in 0.169s!
2/200: starting sentiment analysis for Genlis_Emigres.txt...
finished in 0.036s!
3/200: starting sentiment analysis for Dorat_Malheurs.txt...
finished in 0.04s!
4/200: starting sentiment analysis for Quesne_Folies.txt...
finished in 0.08s!
5/200: starting sentiment analysis for Abbes_Voyage.txt...
finished in 0.045s!
6/200: starting sentiment analysis for Charriere_Observations.txt...
finished in 0.038s!
7/200: starting sentiment analysis for Loaisel_Comtesse.txt...
finished in 0.04s!
8/200: starting sentiment analysis for Retif_Paysanne.txt...
finished in 0.044s!
9/200: starting sentiment analysis for Arnaud_Selicourt.txt...
finished in 0.035s!
10/200: starting sentiment analysis for Marchadier_Isle.txt...
finished in 0.033s!
11/200: starting sentiment analysis for Diderot_Jacques.txt...
finished in 0.037s!
12/200: starting sentiment analysis for Arnaud_Batilde.txt...
finished in 0.042s!
13/200: star

In [16]:
df

,filename,au-name,au-birth,au-death,title,au-gender,firsted-yr,printSource-yr,form,spelling,...,digitalSource_Date,printSource_title,printSource_author,printSource_pubPlace,printSource_publisher,printSource_date,resp_datacapture,resp_encoding,summary,sentiment
0,Abbes_Voyage,Guillaume d'Abbes de Cabreroles,1718,1802,Voyage dans les espaces,M,1758,1758,autodiegetic,historical,...,2013,"Voyage dans les espaces , par d'Abbes de Cabre...",Abbes de Cabreroles,London,unknown,1758,"double keying by ""Jiangsu"", China","Julia Dudar, Christof Schöch","La chronique de Roger-Pol Droit, à propos de ""...",neutral
1,AndreSerieys_Comte,"Jean-François André, Antoine Sérieys","unknown, 1755","unknown, 1819",Le comte d'A*** ou les aventures d'un jeune vo...,M,1800,1800,autodiegetic,historical,...,"2008, 2008",Le comte d'A*** ou les aventures d'un jeune vo...,"Jean-François André, Antoine Sérieys",Paris,Monory,1800,Münchener Digitalisierungszentrum,Johanna Konstanciak,"A la veille de la présidentielle, ""Le Monde"" d...",positive
2,Anonym_Enfant,Anonymous,unknown,unknown,L'enfant du bordel,U,1800,1800,autodiegetic,modern,...,2019,L'Enfant du bordel,anonymous (attribué à Pigault-Lebrun),Paris,unknown,1800,Wikisource contributors,"Henning Gebhard, Johanna Konstanciak",L'E fils du potentat comme celui du savetier s...,neutral
3,Anonym_Histoire,Anonymous,unknown,unknown,Histoire d'un peuple nouveau,U,1757,1757,heterodiegetic,historical,...,"2014, 2014",Histoire d'un peuple nouveau,anonymous,Londres,aux dépens d'une société de libraires,1757,gallica,Johanna Konstanciak,"Dans sa chronique pour le cahier ""Sport&Forme""...",neutral
4,Anonym_Suzon,Anonymous,unknown,unknown,"Mémoires de Suzon, soeur de Dom Bougre",U,1778,1778,autodiegetic,modern,...,2021,"Mémoires de Suzon sœur de D. B., éd. 1778",anonymous,London,unknown,1778,Wikisource contributors,Julia Dudar,"La chronique de Roger-Pol Droit, à propos de ""...",neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Voltaire_Oreilles,Voltaire,1694,1778,Les oreilles du comte de Chesterfield,M,1775,1877,mixed,modern,...,2022,Les oreilles du comte de Chesterfield,Voltaire,Paris,Garnier,1877,Wikisource contributors,"Henning Gebhard, Johanna Konstanciak","Dans sa chronique du cahier ""Sport&Forme"", le ...",neutral
196,Voltaire_Princesse,Voltaire,1694,1778,La Princesse de Babylone,M,1768,1877,heterodiegetic,modern,...,2019,La Princesse de Babylone,François-Marie Arouet Voltaire,Paris,Garnier,1877,Wikisource contributors,Julia Dudar,"Le Palais du ROI de BABYLONE, au cœur de l'Eup...",neutral
197,Voltaire_Scarmentado,Voltaire,1694,1778,Histoire des voyages de Scarmentado,M,1756,1877,autodiegetic,modern,...,2022,Histoire des voyages de Scarmentado écrite par...,Voltaire,Paris,Garnier,1877,Wikisource contributors,"Henning Gebhard, Johanna Konstanciak","A la veille de la présidentielle, ""Le Monde"" d...",positive
198,Voltaire_Songe,Voltaire,1694,1778,Songe de Platon,M,1756,1877,heterodiegetic,modern,...,2022,Songe de Platon,Voltaire,Paris,Garnier,1877,Wikisource contributors,"Henning Gebhard, Johanna Konstanciak","La chronique de Roger-Pol Droit, à propos de «...",negative
